In [3]:
from torchvision.datasets import CIFAR10
from torchvision import transforms

train_data = CIFAR10(root='data', train=True, download=True, transform=transforms.ToTensor())

100%|██████████| 170498071/170498071 [01:51<00:00, 1531742.80it/s]


Extracting data\cifar-10-python.tar.gz to data
